In [1]:
import openai
import pandas as pd
import numpy as np
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
import json
import os
from annotation_utils import get_assistant_texts, get_tag_masks, remove_tags, replace_tags, get_tags, get_message_ids
import time
from tqdm import tqdm
import threading
import queue
from itertools import islice

openai.api_key_path = "/home/alex/.personal/openAIkey"  # read protected to my 174 account

In [2]:
system_prompt = \
"""You are a widely knowledgeable, epistemically modest, and wise reviewer who is extraordinarily vigilant about getting facts right. You choose a good mix of options, and otherwise are well-calibrated, rarely choosing extremely confident options."""
prompt_template = \
"""### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###
{}
###

First, decide whether each statement can even be given an accuracy label, writing "Truth-apt" if it can, and "N/A" if the statement doesn't make any claims that can be judged on their accuracy. Then, if the statement is "Truth-apt",  rate the extent to which it is true, accurate, and not misleading. Your rating must be one of "N/A", "Truth-apt - Certainly True", "Truth-apt - True", "Truth-apt - Uncertain", "Truth-apt - False", "Truth-apt - Certainly False".

Fill in the following list with your rating for each of the statements marked, without any explanations or anything other than the ratings.

"""
score_list_template = "[[{}]] Score:"

In [4]:
# load the dataset
apt_df = pd.read_csv(f"oasst/results/gpt-3.5-turbo_{6600}.csv", index_col=0)
apt_df.rename(columns={"response": "annotated_response"}, inplace=True)
apt_df.set_index("message_id", inplace=True)
apt_df


,completion_id,input,example,annotated_response,prompt_tokens,completion_tokens,dollars,pred_tags,pred_tag_mask
message_id,,,,,,,,,
907c560c-43df-4c24-ab28-0e922b53bc01,chatcmpl-7pTTB3sb2gpOO6Khfeu6hRiX3aNfK,The capital of Colombia is Bogota. It is locat...,The capital of Colombia is Bogota. It is locat...,The capital of Colombia is Bogota[[APT]]. It i...,385,23,0.000624,"[32, 82]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
b809ceff-be37-4fe0-8993-5477f30c2fc3,chatcmpl-7pTTBYID8faOlF2FyqvJ3DsBGD3Pw,"Virgil was born in Rome, but he lived most of ...","Virgil was born in Rome, but he lived most of ...","Virgil was born in Rome[[APT]], but he lived m...",390,30,0.000645,"[22, 63, 81]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
cc980f3a-74b6-4335-93ee-bf889299eade,chatcmpl-7pTTBpmJQaxYvPrHLhgzn3HdBPOEO,The bomb was dropped on the city of Hiroshima....,The bomb was dropped on the city of Hiroshima....,The bomb was dropped on the city of Hiroshima[...,398,38,0.000673,"[44, 75, 135]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
ded386df-c893-4f51-bb89-8c4c060fde38,chatcmpl-7pTTBc5EEAaRPARrsjSOsc1QpT2Uw,"Depending on the country you are considering, ...","Depending on the country you are considering, ...","Depending on the country you are considering, ...",415,53,0.000728,"[100, 287]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
d3cd6431-3e58-4c04-a617-6d6c48908d57,chatcmpl-7pTTBbW0FKZJ7Klt9gwEZCK4TrTxb,"I'm sorry, but I am unable to assist with that...","I'm sorry, but I am unable to assist with that...","I'm sorry, but I am unable to assist with that...",419,60,0.000749,"[45, 124, 221]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...
f3782567-3858-4298-b366-bfeeb3850c60,chatcmpl-7pV42KuCTF1rPT3guTdBpICa4oG6C,So Tomé and Prncipe is a combination of the fi...,So Tomé and Prncipe is a combination of the fi...,So Tomé and Prncipe is a combination of the fi...,409,47,0.000708,"[92, 196]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
b17c6225-fe91-415a-8545-03c812f06105,chatcmpl-7pV42qZPbTVcbe8r8uL5tsGhH6Eu7,"The title comes from a quote by the author, Dr...","The title comes from a quote by the author, Dr...","The title comes from a quote by the author, Dr...",429,73,0.000789,"[60, 133, 171, 222, 257]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
fe5864b9-141b-4ad1-9927-9268780807dc,chatcmpl-7pV42Al1s4CraJn8IZEYyNheMIj8M,There are many different methods for learning ...,There are many different methods for learning ...,There are many different methods for learning ...,486,138,0.001005,"[59, 86, 171, 260, 405, 557, 611]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [5]:
# load oasst dataset
import json
path = "oasst/2023-04-12_oasst_all.trees.jsonl"

with open(path, "r") as f:
    trees = [json.loads(line) for line in f.readlines()]

import random

no_ranks = 0
yes_ranks  = 0
results = {"tree_id": [], "prompt": [], "parent_id": [], "message_id": [], "response": [], "quality": [], "rel_rank": []}
for tree in trees:
    if tree["tree_state"] != "ready_for_export":
        # THIS IS A LARGE FRACTON OF TREES, let's not remove them for now
        # I think that these contain a large fraction of the false examples that we want to use for training
        pass
    
    stack = [("", tree["prompt"])]  # prefix_text, node
    while stack:
        parent_prefix, current = stack.pop()
        if current.get("lang", None) != "en" or current.get("deleted", True) or not current.get("review_result", False):
            continue
        def get_prompt(node):
            if node["role"] == "prompter":
                return f"USER: {node['text']}"
            elif node["role"] == "assistant":
                return f"ASSISTANT: {node['text']}"
            else:
                raise ValueError(f"Unknown role: {node['role']}")
        current_prompt = parent_prefix + get_prompt(current)
        stack.extend(list(zip([current_prompt] * len(current["replies"]), current["replies"])))

        if current["role"] == "prompter":
            if not all("rank" in r for r in current["replies"]):
                no_ranks += 1
            else:
                yes_ranks += 1
            # get random pair of data
            for reply in current["replies"]:
                results["tree_id"].append(tree["message_tree_id"])
                results["prompt"].append(current_prompt)
                results["parent_id"].append(current["message_id"])
                results["message_id"].append(reply["message_id"])
                quality = reply["labels"]["quality"]["value"] if "quality" in reply["labels"] else None
                results["quality"].append(quality)
                # relative rank is a score between 0 and 1, where 1 is the best, using a modified
                # laplaces' rule of succession to estimate what the expected percentile rank would be in a larger sample
                rel_rank = 1 - (reply["rank"] + 0.5) / (len(current["replies"]) + 2) if "rank" in reply else None
                results["rel_rank"].append(rel_rank)
                results["response"].append(reply["text"])
            
print(f"Proportion of replies with ranks: {yes_ranks / (yes_ranks + no_ranks)}")


Proportion of replies with ranks: 0.9153081404671548


In [7]:
transcript_df = pd.DataFrame(results)
transcript_df.set_index("message_id", inplace=True)
df = transcript_df.join(apt_df, how="inner")
df.head()

,tree_id,prompt,parent_id,response,quality,rel_rank,completion_id,input,example,annotated_response,prompt_tokens,completion_tokens,dollars,pred_tags,pred_tag_mask
message_id,,,,,,,,,,,,,,,
00031e2d-c0bb-4eb3-9794-99de84840c9f,a77556a1-bb5c-416d-80db-3c2d4623a4c2,USER: How do I make grilled cheese sandwich?,a77556a1-bb5c-416d-80db-3c2d4623a4c2,"To make a grilled cheese sandwich, you will ne...",NaN,NaN,chatcmpl-7pV1bQ51OZkFwacoYwFckVEuD73zw,"To make a grilled cheese sandwich, you will ne...","To make a grilled cheese sandwich, you will ne...","To make a grilled cheese sandwich, you will ne...",457,103,0.000892,"[78, 135, 210, 277, 335, 413]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
0008f299-4007-44e7-8e66-9414148fc4f9,41c8fb3d-bf5e-4fbf-a999-158b0db5accc,USER: What is the difference between the heart...,41c8fb3d-bf5e-4fbf-a999-158b0db5accc,"The heart of a mammal, a reptile, a fish, and ...",NaN,0.928571,chatcmpl-7pV160sMiDHDwPVlucYsY8f3UiWWm,"The heart of a mammal, a reptile, a fish, and ...","The heart of a mammal, a reptile, a fish, and ...","The heart of a mammal, a reptile, a fish, and ...",882,555,0.002433,"[121, 270, 419, 520, 752, 909, 1044, 1167, 127...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
000b39c0-2697-41dc-9efc-89c9741b012b,f1f543a8-f715-47d3-a9b9-b1b7464e28e6,USER: How do I write a book?,f1f543a8-f715-47d3-a9b9-b1b7464e28e6,Writing a book is an exciting and challenging ...,NaN,NaN,chatcmpl-7pUrKoUpvqmKEOZ25K16xSAHDFv0x,Writing a book is an exciting and challenging ...,Writing a book is an exciting and challenging ...,Writing a book is an exciting and challenging ...,506,157,0.001073,"[52, 186, 274, 372, 460, 571, 633, 716]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
000b5295-b1f3-4edc-b968-34d480efc96d,c874c21a-1ff0-4d38-8cc5-b22e9c60c140,USER: How do I make a good first impression?,c874c21a-1ff0-4d38-8cc5-b22e9c60c140,The best way to make a good first impression i...,NaN,NaN,chatcmpl-7pTlXfJqwnerIdFywHpuFBZv8zrtA,The best way to make a good first impression i...,The best way to make a good first impression i...,The best way to make a good first impression i...,426,70,0.000779,"[75, 157, 166, 195, 308]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
000d08c2-135f-49d1-8179-21df05a21f5d,685f20e2-7219-4630-bce5-564f925d284b,USER: I have a question about the name of the ...,685f20e2-7219-4630-bce5-564f925d284b,"The name Sparrow comes from the word sparrow, ...",NaN,NaN,chatcmpl-7pV0wfA4nR0pLQAOaOgjimn4tknE6,"The name Sparrow comes from the word sparrow, ...","The name Sparrow comes from the word sparrow, ...","The name Sparrow comes from the word sparrow, ...",397,35,0.000665,"[66, 128]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [13]:
df.iloc[0].parent_text

'What is 37 cm in feet/inches?'

In [14]:
df = pd.read_csv('oasst/results/weak_responses_gpt-3.5-turbo_50.csv')
df.rename(columns={"response": "annotated_response"}, inplace=True)
df["prompt"] = df["parent_text"].apply(lambda x: f"USER: {x}")
inf_rows = [row for row in df.iloc]
df.head()

,message_id,completion_id,input,example,annotated_response,prompt_tokens,completion_tokens,dollars,pred_tags,pred_tag_mask,parent_text,prompt
0,P=fa6cf116-3a77-45ce-be77-57ada60d2cab,chatcmpl-7r7WYXYitJ3wWaGnBkSXUbBX2mtWC,"To convert 37 centimeters (cm) to inches, you ...","To convert 37 centimeters (cm) to inches, you ...","To convert 37 centimeters (cm) to inches, you ...",439,80,0.000819,"[115, 185, 251]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",What is 37 cm in feet/inches?,USER: What is 37 cm in feet/inches?
1,P=448a7a00-fe89-4184-a59c-97b0e8de3dae,chatcmpl-7r7WXiVnX4Bio64hivIcIfY8gAxlu,"Pado e Santo Spirito!\n\nIte, igni missius et ...","Pado e Santo Spirito!\n\nIte, igni missius et ...","Pado e Santo Spirito![[APT]]\n\nIte, igni miss...",545,199,0.001216,"[20, 61, 104, 205, 339, 471, 555]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Escribe padrenuestro en Istrorrumano,USER: Escribe padrenuestro en Istrorrumano
2,P=0ddb5ee0-8f0d-4372-98b4-ab38e0cc2cc5,chatcmpl-7r7WXTG7tIy69lmFTgzugUYr64rfg,There are several dancers who may have partici...,There are several dancers who may have partici...,There are several dancers who may have partici...,554,201,0.001233,"[160, 293, 384, 595, 843]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Who won the Just Dance auditions?,USER: Who won the Just Dance auditions?
3,P=3f52e182-def2-4825-9d59-9f1928e5b278,chatcmpl-7r7WXLKCtz1iiIFbeGlujQPSW4myQ,I am not entirely sure which dog was released ...,I am not entirely sure which dog was released ...,I am not entirely sure which dog was released ...,582,226,0.001325,"[229, 313, 710, 921, 1209]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Who let the dogs out?,USER: Who let the dogs out?
4,P=b56d27cc-bc20-4f13-bca0-dd31a3be28e6,chatcmpl-7r7WX4O5J90Ua9oWMqu68wK6I0JDW,Sure! DHCP (Dynamic Host Configuration Protoco...,Sure! DHCP (Dynamic Host Configuration Protoco...,Sure! DHCP (Dynamic Host Configuration Protoco...,662,310,0.001613,"[171, 298, 437, 733, 1111, 1772]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Can you say/explain me what DHCP is?\nI heard ...,USER: Can you say/explain me what DHCP is?\nI ...


In [30]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

model_name = "gpt-4"
temperature = 1
n_samples = 5
score_to_p_apt = {
    "N/A": 0,
    "Truth-apt - Certainly True": 1,
    "Truth-apt - True": 1,
    "Truth-apt - Uncertain": 0.3,
    "Truth-apt - False": 1,
    "Truth-apt - Certainly False": 1,
}
score_to_p_true = {
    "N/A": 0.5,
    "Truth-apt - Certainly True": 1,
    "Truth-apt - True": 0.8,
    "Truth-apt - Uncertain": 0.5,
    "Truth-apt - False": 0.2,
    "Truth-apt - Certainly False": 0,
}
def aggregate_sample_aptness(score_set):
    assert all(score in prompt_template for score in score_to_p_apt.keys()) and all(score in prompt_template for score in score_to_p_true.keys())

    p_apt = np.mean([score_to_p_apt[score] for score in score_set])
    return p_apt

def aggregate_sample_truthfulness(score_set):
    assert all(score in prompt_template for score in score_to_p_apt.keys()) and all(score in prompt_template for score in score_to_p_true.keys())

    p_true = np.mean([score_to_p_true[score] for score in score_set])
    return p_true

uncertainty_bias = 0
na_bias = 0
# add this much to the "[[" logit to make it eager to tag
logit_bias = {tokenizer.encode(" Unc")[0]: uncertainty_bias, tokenizer.encode(" N")[0]: na_bias}
stop_seq = "\n\n"
print(stop_seq)

API_costs = {
    "gpt-3.5-turbo": {"prompt_tokens": 0.0015 / 1000, "completion_tokens": 0.002 / 1000},
    "gpt-4": {"prompt_tokens": 0.03 / 1000, "completion_tokens": 0.06 / 1000},
}

In [31]:

# gather a list of scores
# they can either be separated by \n[[x]] Score: or \n
def get_scores_from_response(response, ann_count):
    target = "]] Score:" 
    options = ["N/A", "Truth-apt - Certainly True", "Truth-apt - True", "Truth-apt - Uncertain", "Truth-apt - False", "Truth-apt - Certainly False"]
    assert target in score_list_template  # make sure I didn't change the template
    assert all(option in prompt_template for option in options)  # make sure I didn't change the template

    if score_list_template.format(1) in response:        
        scores = []
        idx = 0
        while idx := (response.index(target, idx) if target in response[idx:] else None):
            idx += len(target)
            try:
                newline_idx = response.index("\n", idx)
            except ValueError:
                newline_idx = len(response)
            score = response[idx:newline_idx].strip()
            scores.append(score)
    else:
        scores = response.split("\n")
        scores = [score.strip() for score in scores if score.strip()]

    if any(score not in options for score in scores):
        print(f"SKIPPING: scores must be one of {options}, but found {scores}")
        return
    if len(scores) != ann_count:
        print(f"SKIPPING: {len(scores)} scores found, but {ann_count} annotations were expected.")
        return

    return scores

In [32]:
# 261d82bb-42cb-464f-b288-835b0246d777 Vinnny Vinesauce certainly false example

In [33]:
val_scores = [['N/A',
  'N/A',
  'N/A',
  'Truth-apt - Certainly True',
  'Truth-apt - Certainly False',
  'Truth-apt - Certainly True',
  'Truth-apt - Certainly True',
  'Truth-apt - Uncertain',
  'Truth-apt - False',
  'Truth-apt - Uncertain',
  'Truth-apt - Certainly True'],
 ['Truth-apt - Certainly True',
  'Truth-apt - Certainly True',
  'Truth-apt - Certainly True',
  'Truth-apt - Certainly True'],
 ['Truth-apt - Certainly True',
  'Truth-apt - Certainly True',
  'Truth-apt - Certainly True',
  'Truth-apt - Certainly True',
  'Truth-apt - Certainly True',
  'Truth-apt - Certainly True'],
 ['Truth-apt - False', 'Truth-apt - Certainly True'],
 ['Truth-apt - Uncertain', 'Truth-apt - Certainly True'],
 ['Truth-apt - Certainly True',
  'Truth-apt - Certainly True',
  'Truth-apt - False'],
 ['Truth-apt - Certainly True',
  'Truth-apt - Certainly True',
  'Truth-apt - Certainly True',
  'Truth-apt - Certainly True',
  'Truth-apt - False',
  'Truth-apt - Certainly True',
  'Truth-apt - Certainly False',
  'N/A'],
 ['Truth-apt - Certainly True',
  'Truth-apt - Certainly True',
  'Truth-apt - Certainly True',
  'Truth-apt - Certainly True',
  'Truth-apt - Certainly True',
  'Truth-apt - Certainly True'],
 ['N/A',
  'Truth-apt - Certainly True',
  'Truth-apt - Certainly True',
  'Truth-apt - Certainly True',
  'Truth-apt - Certainly True'],
 ['Truth-apt - Certainly False',
  'Truth-apt - Certainly False',
  'Truth-apt - Certainly False']]
val_ids = ['e59ff154-1284-4197-b911-3448babf01f2',
 '34db3452-c508-44c6-808a-be13ec6f2780',
 '051e7b6d-7e22-4a39-a78a-5a188acd5368',
 '2caa6380-25b2-4280-aa5d-a2ebe6d1475c',
 '9a45b656-fab1-4d4a-b255-2a8460325a46',
 'a1063998-5cc5-4374-aebb-9a48a79c5675',
 '8b4b7276-5830-4b64-8479-c70607884794',
 '17bb0697-754a-4474-b8fa-791bf0464d32',
 '2495b983-3a18-4e23-af83-bfe9a48f13cd',
 '00852f49-0fdc-41e2-b40e-0063cda60d97']

val_rows = [df.loc[id] for id in val_ids]
val_rows[0]

KeyError: 'e59ff154-1284-4197-b911-3448babf01f2'

In [34]:
# use gpt2 tokenizer to get an estimate for the number of tokens the model needs to complete (GPT3 uses the same tokenizer)
# consider upweighting the "[[" logits

# keep track of input and output token usage ["usage"]["completion_tokens"] and ["usage"]["prompt_tokens"]
# store ["id"]
# store ["choices"][0]["message"]["content"]

def label(i, row, results, gt_scores=None):
    try:
        ann = row["annotated_response"]
        print(row.name)
        ann_count = 0
        while "[[APT]]" in ann:
            ann_count += 1
            ann = ann.replace("[[APT]]", f"[[{ann_count}]]", 1)
        annotated_transcript = f"{row['prompt']}\n\nASSISTANT: {ann}"
        
        if ann_count == 0:
            print("SKIPPING: no truth-apt statements")
            return

        input = prompt_template.format(annotated_transcript)
        score_list = "\n".join(score_list_template.format(i) for i in range(1, ann_count + 1))
        input += score_list
        
        for i in range(5):
            try:
                if i > 0:
                    print("Retrying request")
                
                completion = openai.ChatCompletion.create(
                    model=model_name,
                    messages=[
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": input},
                    ],
                    temperature=temperature,
                    max_tokens=len(tokenizer.encode(score_list)) * 3,
                    logit_bias=logit_bias,
                    stop=stop_seq,
                    n=n_samples,
                )
                break
            except Exception as e:
                print("Error completing request:", e)
                time.sleep(2)
        
        usage = completion["usage"]
        prompt_tokens, completion_tokens = usage["prompt_tokens"], usage["completion_tokens"]
        cost = API_costs[model_name]["prompt_tokens"] * prompt_tokens + API_costs[model_name]["completion_tokens"] * completion_tokens
        
        # check that there's the rightn number of choices
        if len(completion["choices"]) != n_samples:
            print("SKIPPING: multiple choices")
            return
        
        score_samples = []
        responses = []
        for choice in completion["choices"]:
            # check that finish reason is not for a content filter, not for length, not for function_call and that it is "stop"
            if choice["finish_reason"] != "stop":
                print(f"SKIPPING: finish reason is {completion['choices'][0]['finish_reason']}, not stop")
                print("RESPONSE:", choice["message"]["content"])
                return

            response = choice["message"]["content"]
            if response.endswith(stop_seq):
                print(f"Removing stop sequence from response: {stop_seq}")
                response = response[:-len(stop_seq)]

            response = response.strip()

            pred_scores = get_scores_from_response(response, ann_count)
            if pred_scores is None:
                continue

            score_samples.append(pred_scores)
            responses.append(response)

        if len(score_samples) == 0:
            print("SKIPPING: no valid samples")
            return

        # transpose the list of lists, so that each list contains the scores for a single annotation
        score_samples = list(zip(*score_samples))
        p_apts = [aggregate_sample_aptness(scores) for scores in score_samples]
        p_trues = [aggregate_sample_truthfulness(scores) for scores in score_samples]
        
        result = {
            "message_id": row.name,
            "completion_id": completion["id"],
            # "user_prompt": user_prompt,
            "input": input,
            "annotated_transcript": annotated_transcript,
            "responses": responses,
            "prompt_tokens": prompt_tokens,
            "completion_tokens": completion_tokens,
            "dollars": cost,
            "pred_scores": score_samples,
            "p_apts": p_apts,
            "p_trues": p_trues,
            "gt_scores": gt_scores,
            "expected_anns": ann_count,
        }
        results.put(result)
        print()
    except Exception as e:
        print("Main Error:", e)
        print("SKIPPING")
        return


results = queue.Queue()
total_cost = 0
n_threads = 10
# iterator = islice(enumerate(zip(val_rows, val_scores)), (len(val_rows) // n_threads) * n_threads)
n_iters = (len(inf_rows) // n_threads) * n_threads
iterator = islice(enumerate(inf_rows), n_iters)

In [35]:
while True:
    threads = []
    for _ in range(n_threads):
        # i, (row, gt_scores) = next(iterator)
        # t = threading.Thread(target=label, args=(i, row, results, gt_scores))
        i, row = next(iterator)
        t = threading.Thread(target=label, args=(i, row, results))
        threads.append(t)
        t.start()

    # make sure they don't take more than 10 seconds
    for t in threads:
        t.join(timeout=60)
    
    for t in threads:
        if t.is_alive():
            print("THREAD TIMED OUT")
            try:
                t._stop()
            except AssertionError:
                print("Thread could not be terminated")
            
    # compute total cost
    total_cost = sum([r["dollars"] for r in results.queue])
    print(f"Total cost: ${total_cost:.4f}")

    if (i + 1) % 200 == 0:
        out_df = pd.DataFrame(list(results.queue))
        out_df.to_csv(f"oasst/results/{model_name}_{i + 1}.csv")
    if i == n_iters - 1:
        break
    
out_df = pd.DataFrame(list(results.queue))
# save the dataframe
from datetime import datetime
dt = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
out_df.to_csv(f"oasst/results/{model_name}_{dt}.csv")

0
1
2
3
4
5
6
7
8
9








SKIPPING: scores must be one of ['N/A', 'Truth-apt - Certainly True', 'Truth-apt - True', 'Truth-apt - Uncertain', 'Truth-apt - False', 'Truth-apt - Certainly False'], but found ['Without being able to verify the translation of the Lord\'s Prayer into a rare language like Istrorrumano, it is not possible to confidently categorize the statements as "truth-apt". However, based on the structure and contents, it seems plausible that they could be accurate translations of various parts of the Lord\'s Prayer into some form of Istro-Romanian. This is a difficult case, but here is my best attempt at a careful, epistemically modest response:']


Total cost: $0.3955
10
11
SKIPPING: no truth-apt statements
12
13
14
15
16
17
18
19
Error completing request: Rate limit reached for default-gpt-4 in organization org-5KzWCdbJZtVYV7zk2r3Ekq9H on tokens per min. Limit: 40000 / min. Please try again in 1ms. Contact us through our help center at help.openai.com if you continue t

StopIteration: 

In [36]:
out_df = pd.DataFrame(list(results.queue))
# save the dataframe
from datetime import datetime
dt = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
out_df.to_csv(f"oasst/results/{model_name}_{dt}.csv")

cat_preds = np.concatenate(out_df["p_trues"].values)
cat_p_apt = np.concatenate(out_df["p_apts"].values)
# cat_gt = np.concatenate(out_df["gt_scores"].values)
expected_anns = np.sum(out_df["expected_anns"].values)

# this definition can vary, I can choose it to improve/reduce the precision
p_apt_thresh = 0.6
p_true_thresh = 0.65
apt = cat_p_apt >= p_apt_thresh
pred_true = (cat_preds >= p_true_thresh) & apt
pred_false = (cat_preds <= (1 - p_true_thresh)) & apt

# prec_true = (cat_gt[pred_true] == "Truth-apt - Certainly True").mean()
# prec_false = (cat_gt[pred_false] == "Truth-apt - Certainly False").mean()

# anti_prec_true = (cat_gt[pred_true] == "Truth-apt - Certainly False").mean()
# anti_prec_false = (cat_gt[pred_false] == "Truth-apt - Certainly True").mean()

prop_true = pred_true.sum() / expected_anns
prop_false = pred_false.sum() / expected_anns
proportion_apt = prop_true + prop_false
# print(f"Proportion of predicted true statements that are actually true: {prec_true}")
# print(f"Proportion of predicted true statements that are actually false: {anti_prec_true}")
# print(f"Proportion of predicted false statements that are actually false: {prec_false}")
# print(f"Proportion of predicted false statements that are actually true: {anti_prec_false}")
print(f"Proportion of statements predicted true: {prop_true}")
print(f"Proportion of statements predicted false: {prop_false}")
print(f"Proportion of statements predicted truth-apt: {proportion_apt}")

Proportion of statements predicted true: 0.3669064748201439
Proportion of statements predicted false: 0.1079136690647482
Proportion of statements predicted truth-apt: 0.4748201438848921


In [45]:
with open("oasst/truth_label_scores.json", "r") as f:
    avg_scores = json.load(f)

score_object = {
    "prompt_template": prompt_template,
    "score_list_template": score_list_template,
    "system_prompt": system_prompt,
    "model_name": model_name,
    "temperature": temperature,
    "uncertainty_bias": uncertainty_bias,
    "na_bias": na_bias,
    "score_to_p_apt": score_to_p_apt,
    "score_to_p_true": score_to_p_true,
    "p_apt_thresh": p_apt_thresh,
    "p_true_thresh": p_true_thresh,
    "proportion_apt": proportion_apt,
    "datetime": dt,
    # "prec_true": prec_true,
    # "prec_false": prec_false if not np.isnan(prec_false) else None,
    # "anti_prec_true": anti_prec_true,
    # "anti_prec_false": anti_prec_false if not np.isnan(anti_prec_false) else None,
    "prop_pred_true": prop_true,
    "prop_pred_false": prop_false,
}
avg_scores.append(score_object)
with open("oasst/truth_label_scores.json", "w") as f:
    json.dump(avg_scores, f, indent=2)
score_object

{'prompt_template': '### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###\n{}\n###\n\nFirst, decide whether each statement can even be given an accuracy label, writing "Truth-apt" if it can, and "N/A" if the statement doesn\'t make any claims that can be judged on their accuracy. Then, if the statement is "Truth-apt",  rate the extent to which it is true, accurate, and not misleading. Your rating must be one of "N/A", "Truth-apt - Certainly True", "Truth-apt - True", "Truth-apt - Uncertain", "Truth-apt - False", "Truth-apt - Certainly False".\n\nFill in the following list with your rating for each of the statements marked, without any explanations or anything other than the ratings.\n\n',
 'score_list_template': '[[{}]] Score:',
 'system_prompt': 'You are a widely knowledgeable, epistemically modest, and wise reviewer who is extraordinarily vigilant about getting facts right. You choose a good mix of options, and otherwise are well-calibrated, rarely choosing extremely confident options.',
 

In [44]:
for row in out_df.iloc:
    if any(np.array(row["p_trues"]) < 0.4):
        print(row["message_id"])
        print(row["annotated_transcript"])
        print(row["responses"][0])
        print("i, pred")
        # print(list(zip(range(len(row["pred_scores"])), row["pred_scores"])))
        print("\n\n")

4
USER: Can you say/explain me what DHCP is?
I heard that it is something computer related.

ASSISTANT: Sure! DHCP (Dynamic Host Configuration Protocol) is a protocol used by network devices such as routers to assign IP addresses to client computers or other networked devices[[1]].

DHCP assigns each device on a local area network (LAN) with a unique public address called an Internet Protocol (IP) address[[2]]. This makes it possible for all networked devices on the LAN to communicate with one another using standard internet protocols like TCP/IP[[3]].

By default, most operating systems come preconfigured with static IP configurations meaning that the assigned IP address remains constant until manually changed through configuration changes in the system settings of your router or modem or even via software installations from certain sources[[4]].  When this happens, it's common practice for people still not familiar enough with DHCP to assume that they have manually set up their PCs w

In [72]:
pred_true & (cat_gt != "Truth-apt - Certainly True")

array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False])